In [72]:
# IMPORTS
import pandas as pd

import tkinter as tk
from tkinter import filedialog as fd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

import re
from unidecode import unidecode

from datetime import datetime

In [73]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [74]:
# funcion para ejecutar una script SQL
# estado = 0 para select
# estado = 1 para update
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [75]:
# GENERAR CODIGO SLUG, CODIGO UNICO
def abbreviate_institution_name(name):
    # Eliminar acentos y otros caracteres especiales
    name = unidecode(name)
    # Mantener solo caracteres alfanuméricos, espacios y guiones
    name = re.sub(r"[^\w\s-]", "", name)
    # Convertir a minúsculas
    name = name.lower()
    # Reemplazar espacios y guiones por un solo guión
    abbreviated = re.sub(r"[\s-]+", "-", name)
    return abbreviated.strip('-')

In [77]:
# OPEN FILE DIALOG
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()
file_name

'D:/OneDrive - Ingenio Azucarero Guabira S.A/APP CAÑERO/DATA PARA CARGAR/INSTITUCIONES.xlsx'

In [78]:
data = pd.read_excel(file_name)
data.head(3)

,cod_inst,nom_inst,telf,direccion,cell,obs
0,1,1 - ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑ...,9220257,C/ Rosendo Paz Nº 148,NaN,NaN
1,3,3 - ASOCIACIÓN AGROPECUARIA 21 DE ABRIL,9221774,C/ Rosendo Paz Nº 146,NaN,NaN
2,5,5 - ASOC. DE CAÑEROS SANTISTEBAN,9220838,Av. Paichane Esq. Evaristo Mendez,NaN,NaN


In [90]:
data.shape

(40, 6)

In [87]:
len(data)

40

In [103]:
for i in range(len(data)):
    fila = data.iloc[i]
    
    # optiene la fecha y formatear
    now = datetime.now()
    formatted_now = now.strftime("%Y-%m-%d %H:%M:%S")
    
    # VERIFICAR QUE LA INST AUN NO ESTE REGISTRADO
    # seleccionar el codigo de inst
    select = 'select * from institucion where codigo=\'{inst}\''.format(inst=fila['cod_inst'])
    # ejecutar seleccion
    select_inst = dbQuery(select)
    # si es mayor a cero, la inst ya esta registrada
    if len(select_inst) > 0:
        print('Ya registrado:', select_inst[0]['nombre'])
        continue
    query = 'insert into institucion (codigo, nombre, slug, direccion, telefono, estado, created_at, updated_at) values ({codigo}, \'{nombre}\', \'{slug}\',\'{direccion}\', \'{telefono}\',\'{estado}\', \'{created_at}\', \'{updated_at}\')'.format(codigo=fila['cod_inst'], nombre=fila['nom_inst'], slug='', direccion=fila['direccion'], telefono=fila['telf'], estado='activo', created_at=formatted_now, updated_at=formatted_now)
    insert_row = dbQuery(query, 1)
    
    # SELECCIONAR LA INST QUE SE ACABA DE INSERTAR PARA OBTENER EL ID DEL REGISTRO
    select = 'select * from institucion where codigo=\'{inst}\''.format(inst=fila['cod_inst'])
    # ejecutar seleccion
    select_inst = dbQuery(select)
    inst = select_inst[0]
    
    # generar el codigo slug
    slug_cod = abbreviate_institution_name(inst['nombre']) + '-' + str(inst['id'])
    # actualizar el codigo slug segun el id de la ultima seleccion
    query = 'update institucion set slug=\'{slug}\' where id={id}'.format(slug=slug_cod, id=inst['id'])
    # ejecutar update
    update_slug = dbQuery(query, 1)
    print('Registrado Inst:', inst['nombre'])

Ya registrado: 1 -  ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑA CAMPESINO C.T.C.
Ya registrado: 3 -  ASOCIACIÓN AGROPECUARIA 21 DE ABRIL
Registrado Inst: 5 -  ASOC. DE CAÑEROS SANTISTEBAN
Ya registrado: 6 -  ASOCIACIÓN CIVIL DE PEQUEÑOS CAÑEROS COLONIA AROMA ACIPECCA
Registrado Inst: 8 -  ASOCIACIÓN DE CAÑEROS GUABIRÁ
Registrado Inst: 18 -  COOPERATIVA AGRICOLA 10 DE NOVIEMBRE LTDA.
Registrado Inst: 19 -  COOPERATIVA 10 DE OCTUBRE LTDA.
Registrado Inst: 30 -  COOPERATIVA AGROPECUARIA 30 DE MAYO LTDA.
Registrado Inst: 32 -  COOPERATIVA AGROPECUARIA ABAROA LTDA.
Registrado Inst: 35 -  COOPERATIVA  AGROPECUARIA EL INCA LTDA.
Registrado Inst: 37 -  COOPERATIVA AGROPECUARIA INTEGRAL INDEPENDENCIA LTDA.
Ya registrado: 40 -  ASOCIACIÓN  AGROPECUARIA GUABIRA
Registrado Inst: 42 -  COOPERATIVA AGROPECUARIA AGUA DULCE LTDA.
Registrado Inst: 43 -  ASOCIACION CIVIL DE PEQUEÑOS CAÑEROS BOLIVAR
Registrado Inst: 48 -  COOPERATIVA  AGRICOLA GUADALQUIVIR LTDA. 
Ya registrado: 50 - ASOCIACIÓN AGROPECUARI

## paso a paso

In [1]:
fila = data.iloc[0]
fila['cod_inst']

NameError: name 'data' is not defined

In [46]:
# optiene la fecha actual
now = datetime.now()
formatted_now = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_now)

2023-05-16 14:43:40


In [70]:
# seleccionar instituacion
select = 'select * from institucion where codigo=\'{inst}\''.format(inst=1)
# ejecutar seleccion
select_inst = dbQuery(select)
select_inst[0]['nombre']

[{'id': 26,
  'codigo': '1',
  'nombre': '1 -  ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑA CAMPESINO C.T.C.',
  'slug': '1-asociacion-de-pequenos-productores-de-cana-campesino-ctc-26',
  'direccion': 'C/ Rosendo Paz Nº 148',
  'telefono': '9220257',
  'estado': 'activo',
  'created_at': datetime.datetime(2023, 5, 16, 14, 43, 40),
  'updated_at': datetime.datetime(2023, 5, 16, 14, 43, 40)}]

In [71]:
len(select_inst)

1

In [49]:
query = 'insert into institucion (codigo, nombre, slug, direccion, telefono, estado, created_at, updated_at) values ({codigo}, \'{nombre}\', \'{slug}\',\'{direccion}\', \'{telefono}\',\'{estado}\', \'{created_at}\', \'{updated_at}\')'.format(codigo=fila['cod_inst'], nombre=fila['nom_inst'], slug='', direccion=fila['direccion'], telefono=fila['telf'], estado='activo', created_at=formatted_now, updated_at=formatted_now)

In [50]:
query

"insert into institucion (codigo, nombre, slug, direccion, telefono, estado, created_at, updated_at) values (1, '1 -  ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑA CAMPESINO C.T.C.', '','C/ Rosendo Paz Nº 148', '9220257','activo', '2023-05-16 14:43:40', '2023-05-16 14:43:40')"

In [51]:
insert_row = dbQuery(query, 1)

In [52]:
# seleccionar instituacion
select = 'select * from institucion where codigo=\'{inst}\''.format(inst=fila['cod_inst'])
# ejecutar seleccion
select_inst = dbQuery(select)

In [53]:
inst = select_inst[0]
inst

{'id': 26,
 'codigo': '1',
 'nombre': '1 -  ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑA CAMPESINO C.T.C.',
 'slug': '',
 'direccion': 'C/ Rosendo Paz Nº 148',
 'telefono': '9220257',
 'estado': 'activo',
 'created_at': datetime.datetime(2023, 5, 16, 14, 43, 40),
 'updated_at': datetime.datetime(2023, 5, 16, 14, 43, 40)}

In [54]:
slug_cod = abbreviate_institution_name(inst['nombre']) + '-' + str(inst['id'])
slug_cod

'1-asociacion-de-pequenos-productores-de-cana-campesino-ctc-26'

In [55]:
query = 'update institucion set slug=\'{slug}\' where id={id}'.format(slug=slug_cod, id=inst['id'])

In [56]:
query

"update institucion set slug='1-asociacion-de-pequenos-productores-de-cana-campesino-ctc-26' where id=26"

In [57]:
update_slug = dbQuery(query, 1)

In [58]:
update_slug

()